# Machine Learning: K supercomputer double perovskite dataset

## Part 2: machine learning

Welcome to this tutorial! This notebook follows from a lecture given at Cal State LA from the weeks of May 20 to May 24, 2019.

In Part 1, we looked at how we can generate atomic features for a double perovskite data set. Next, we will try to see if we can predict the band gap using a machine learning algorithm called a *random forest*.

**Important!** The code from this notebook was run live during a lecture. The hyperparameter tuning is **very** minimal; one can certainly do much better than this. Moreover, there are other machine learning algorithms that perform better than a random forest, such as XGBoost. However, I kept it simple so that the audience would be familiarized with scikit-learn and it would run quickly when doing a live demo.

Let's import `numpy`, `pandas`, and `sklearn` (the package containing the actual random forest algorithm):

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

Let's read in the data frame we worked on last time:

In [2]:
df = pd.read_pickle('df_new.pkl')

Let's make sure we are dealing with numerical values.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1921 entries, 50 to 9847
Data columns (total 33 columns):
System                            1921 non-null object
LowGap_SOHSE_Approx               1921 non-null float64
A_sites                           1921 non-null object
B_sites                           1921 non-null object
B_prime_sites                     1921 non-null object
X_sites                           1921 non-null object
B_IonicRadius_sum                 1921 non-null object
B_IonicRadius_dif                 1921 non-null object
X_IonicRadius                     1921 non-null object
B_Electronegativity_sum           1921 non-null object
B_Electronegativity_dif           1921 non-null object
X_Electronegativity               1921 non-null object
B_BoilingPoint_sum                1921 non-null object
B_BoilingPoint_dif                1921 non-null object
X_BoilingPoint                    1921 non-null object
B_MeltingPoint_sum                1921 non-null object
B_MeltingPo

It looks like the desired features are not floats, so we need to convert them. Let's use `numpy,float64`.

In [4]:
for col in df.columns[6:]:
    df[col] = df[col].astype(np.float64)

Let's make sure this worked:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1921 entries, 50 to 9847
Data columns (total 33 columns):
System                            1921 non-null object
LowGap_SOHSE_Approx               1921 non-null float64
A_sites                           1921 non-null object
B_sites                           1921 non-null object
B_prime_sites                     1921 non-null object
X_sites                           1921 non-null object
B_IonicRadius_sum                 1921 non-null float64
B_IonicRadius_dif                 1921 non-null float64
X_IonicRadius                     1921 non-null float64
B_Electronegativity_sum           1921 non-null float64
B_Electronegativity_dif           1921 non-null float64
X_Electronegativity               1921 non-null float64
B_BoilingPoint_sum                1921 non-null float64
B_BoilingPoint_dif                1921 non-null float64
X_BoilingPoint                    1921 non-null float64
B_MeltingPoint_sum                1921 non-null float64
B

Let's use all the columns that contain numerical data (besides the band gap we which to predict), and use this as our $\boldsymbol{X}$, in which rows represent individual data observations, and columns represent features. In machine learning, this is sometimes called a *design matrix*. By convention, matrices are written as capital letters, so we will use a capital `X` in code:

In [6]:
X = df[df.columns[6:]]

Let's set our $\boldsymbol{y}$ as a vector containing all of the band gaps. This is sometimes called a *target vector*. Let's check the shapes of our $\boldsymbol{X}$ and $\boldsymbol{y}$:

In [7]:
y = df['LowGap_SOHSE_Approx']

X.shape, y.shape

((1921, 27), (1921,))

Now, let's use the function `train_test_split` to make splits for training and testing. We will use a test set size of $0.2$. The `random_state=42` ensures that the same random split is returned every time (for the sake of reproducibility).

In [8]:
# Returns four numbers:
# training X, test X,
# training y, test y

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2,
                                      random_state=42)

Xtr.head()

,B_IonicRadius_sum,B_IonicRadius_dif,X_IonicRadius,B_Electronegativity_sum,B_Electronegativity_dif,X_Electronegativity,B_BoilingPoint_sum,B_BoilingPoint_dif,X_BoilingPoint,B_MeltingPoint_sum,...,X_Rp,B_Rd_sum,B_Rd_dif,X_Rd,B_FirstIonizationPotential_sum,B_FirstIonizationPotential_dif,X_FirstIonizationPotential,A_IonicRadius,ToleranceFactor_sum,ToleranceFactor_dif
2332,2.2725,0.3925,0.882500,3.40,0.16,2.96,4091.00,599.0,332.00,1006.75,...,0.62,0.823,0.103,0.143,11.8947,0.3219,11.8138,2.17,2.158798,0.209865
803,1.5900,0.0700,1.273333,3.86,0.24,2.66,4337.00,617.0,457.40,1206.69,...,0.83,0.505,0.165,0.315,14.6393,2.6407,10.4513,2.17,2.355038,0.039852
6388,1.6300,0.0900,0.780000,3.32,0.70,3.16,4456.00,1730.0,239.11,2134.40,...,0.51,0.160,0.160,0.000,15.5462,0.2538,12.9676,2.53,2.937164,0.082867
5236,1.3700,0.2900,0.882500,3.86,0.06,2.96,6048.00,298.0,332.00,2192.08,...,0.62,0.345,0.345,0.143,15.4955,0.8079,11.8138,2.53,3.105363,0.287258
1270,2.4900,0.0000,0.882500,4.00,0.00,2.96,1259.76,0.0,332.00,468.64,...,0.62,0.950,0.000,0.143,20.8750,0.0000,11.8138,2.17,2.029089,0.000000


Let's check the shapes of the $\boldsymbol{X}$ and $\boldsymbol{y}$ for our splits:

In [9]:
Xtr.shape, Xte.shape, ytr.shape, yte.shape

((1536, 27), (385, 27), (1536,), (385,))

Let's create a simple random forest object. We will use a random state as well, since there is some element of randomness in how the random forest works. Notice how there are some hyperparameters set as default; we will try to adjust these later.

In [10]:
rfr = RandomForestRegressor(random_state=42, n_jobs=-1)

rfr

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

Let's train our regressor on the training data:

In [11]:
rfr.fit(Xtr, ytr)

/Users/kawazu/.anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

Let's look at the mean squared error and mean absolute error. Note that the random forest regressor seeks to decrease mean squared error.

*Side note:* mean squared error punishes large errors more harshly than many smaller errors.

In [12]:
mean_squared_error(yte, rfr.predict(Xte))

0.28419785065471403

In [13]:
mean_absolute_error(yte, rfr.predict(Xte))

0.3793655412987013

Next, we will try different values for hyperparameters that we took as default in the beginning. The `RandomizedSearchCV` object from scikit-learn takes in a dictionary, where the key is the hyperparameter that is to be tuned, and the value is a list of the possible values to try. It will perform [$k$-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation) to rank which configuration performs the best.

In [14]:
from sklearn.model_selection import RandomizedSearchCV

Let's make the dictionary of hyperparameters:

In [15]:
# Hyperparameter dictionary
# Keys: hyperparameter you want to tune
# Values: list of the values you want to try

hp_dict = {
    'n_estimators': [int(i) for i in np.linspace(50, 1000, 20)],
    'max_depth': [int(i) for i in np.linspace(10, 100, 10)] + [None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 'sqrt']    
}

hp_dict

{'n_estimators': [50,
  100,
  150,
  200,
  250,
  300,
  350,
  400,
  450,
  500,
  550,
  600,
  650,
  700,
  750,
  800,
  850,
  900,
  950,
  1000],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_split': [2, 5, 10, 15, 20],
 'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 'sqrt']}

Let's make a `RandomizedSearchCV` object. We need to specify the algorithm (`estimator`). Let's set this to the random forest `rfr` we set earlier. The grid of configurations to try (`param_distributions`) will be the `hp_dict` dictionary we created. `n_iter` tests how many combinations we wish to try. Here it is `3` to run it quickly for demonstration purposes. `cv` dictates the $k$ value for $k$-fold cross validation. We use `random_state=42` to ensure reproducibility. `verbose=3` makes a lot of the information print, which is convenient in tracking the progress. Finally, `n_jobs=-1` indicates that all cores will be used.

**Important!** The value of `n_iter` is far too small for real-life applications. The value was chosen to be small to run slowly in the live lecture. Look for values in the neighborhood of hundreds or thousands. You also may wish to look into `sklearn.model_selection.GridSearchCV`, which tests exhaustively each combination of hyperparameters.

**Important!** `n_jobs` can also be set for the `RandomForestRegressor` itself. The `rfr` object by default uses one core. However, regardless of the number of processors used for the `RandomForestRegressor`, it will train in parallel as many `rfr` objects as there are cores. If you set `n_jobs=-1` in the `RandomSearchCV` and `n_jobs` greater than 1 in the `RandomForestRegressor`, you may risk overloading your CPUs, leading to poor performance.

In [16]:
rand = RandomizedSearchCV(
    estimator=rfr,
    param_distributions=hp_dict,
    n_iter=3,
    cv=3,
    random_state=42,
    verbose=3,
    n_jobs=-1
)

Let's fit our `RandomizedSearchCV` object:

In [17]:
rand.fit(Xtr, ytr)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    2.4s remaining:    8.4s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    3.7s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    4.0s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=3, n_jobs=-1,
          param_distributions={'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'min_samples_split': [2, 5, 10, 15, 20], 'max_features': [0.1, 0.2, 0.3, 0.4, 0.5, 'sqrt']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=3)

What were the best hyperparameters? (By **no means** is this an exhaustive search.) Let's find out:

In [18]:
rand.best_params_

{'n_estimators': 350,
 'min_samples_split': 5,
 'max_features': 0.5,
 'max_depth': 90}

Let's create a `RandomForestRegressor` object with the best hyperparameters from our search, and look at its performance:

In [19]:
rand.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=90,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

What was its mean squared error?

In [20]:
mean_squared_error(yte, rand.best_estimator_.predict(Xte))

0.25410944822395176

What was its mean absolute error?

In [21]:
mean_absolute_error(yte, rand.best_estimator_.predict(Xte))

0.3637732862905896

We can also see which columns (features) were most important for the random forest algorithm. This can help us whittle down the number of features in case they are too many features, and also help us interpret the model.

In [22]:
rand.best_estimator_.feature_importances_

array([0.02928105, 0.02071222, 0.0174927 , 0.20966352, 0.05428223,
       0.01836623, 0.04117425, 0.05902698, 0.01840578, 0.04581402,
       0.0371618 , 0.01642706, 0.03121912, 0.02422141, 0.0173211 ,
       0.03523402, 0.0466566 , 0.01823948, 0.02969565, 0.01776696,
       0.01971224, 0.06552189, 0.03108234, 0.02006746, 0.01059797,
       0.03402934, 0.03082659])

Finally, let's look at the names that these importances correspond to. The `sorted` with `key=lambda x: -x[1]` allows us to sort by the negative of the second element in some tuple; thus, the largest importance will be in the front of the list. One could also use `reverse=True` instead of the negative sign.

In [23]:
sorted(list(zip(X.columns, rand.best_estimator_.feature_importances_)),
       key=lambda x: -x[1])

[('B_Electronegativity_sum', 0.2096635207097799),
 ('B_FirstIonizationPotential_sum', 0.06552189392073311),
 ('B_BoilingPoint_dif', 0.059026982877728965),
 ('B_Electronegativity_dif', 0.05428223256676176),
 ('B_Rp_dif', 0.04665659772673598),
 ('B_MeltingPoint_sum', 0.04581401504826196),
 ('B_BoilingPoint_sum', 0.04117425020310157),
 ('B_MeltingPoint_dif', 0.03716179672088477),
 ('B_Rp_sum', 0.035234024358767144),
 ('ToleranceFactor_sum', 0.03402933692221601),
 ('B_Rs_sum', 0.031219122180312232),
 ('B_FirstIonizationPotential_dif', 0.031082343246673656),
 ('ToleranceFactor_dif', 0.030826587092878256),
 ('B_Rd_sum', 0.02969565455536849),
 ('B_IonicRadius_sum', 0.02928104587712013),
 ('B_Rs_dif', 0.02422141260885283),
 ('B_IonicRadius_dif', 0.020712218265346707),
 ('X_FirstIonizationPotential', 0.020067456887935153),
 ('X_Rd', 0.01971224159252183),
 ('X_BoilingPoint', 0.018405775291338862),
 ('X_Electronegativity', 0.018366227753744024),
 ('X_Rp', 0.0182394806870847),
 ('B_Rd_dif', 0.0177

Thank you for your time and attention in this workshop!